# Compulsory assignment 1 - Using Dask-ml on large data
## Group #16
### Thomas Moen and Jørgen Kongsro

In [1]:
%matplotlib notebook


import numpy as np
import pandas as pd
import dask
import os


from operator import itemgetter

import numpy as np
import scipy

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import matplotlib.pyplot as plt
import skimage.io

from dask_ml.linear_model import LogisticRegression
from dask_ml.model_selection import train_test_split


In [ ]:
# Install Kaggle API
# How to setup: https://github.com/Kaggle/kaggle-api
# or visit: https://adityashrm21.github.io/Setting-Up-Kaggle/

#!pip install kaggle

In [ ]:
# Download Kaggle data using Kaggle API
#!kaggle competitions download -c dat300-ca1-autumn-2019


In [ ]:
# Unzip Kaggle data
#!unzip "dat300-ca1-autumn-2019.zip" -d "/tmp/whatever"

In [2]:
# List files in directory

if os.name == 'nt':
    workdir = 'C://Users//thomoe//Documents//myDAT300//dat300-ca1-autumn-2019//'
elif os.name == 'posix':
    workdir = '/Users/jorgenkongsro/Downloads/dat300-ca1-autumn-2019/'
    
os.listdir(workdir)


PermissionError: [Errno 1] Operation not permitted: '/Users/jorgenkongsro/Downloads/dat300-ca1-autumn-2019/'

In [ ]:
def import_data(csv_file):
    """ Import data from csv file

    :param data: a .csv separated dataset
    :return: a pandas data array, df
    
    """
    
    
    df = dd.read_csv(csv_file)
    return df

x_train_df = import_data(workdir + 'X_train.csv')
x_test_df = import_data(workdir + 'X_test.csv')
y_train_df = import_data(workdir + 'y_train.csv')

x_test_df.head()

In [ ]:
def percent_missing(dataframe):
    """ Check for percent missing values in dataframe
    :param data: dataframe
    :return: dataframe
    
    """
    missing_values = dataframe.isnull().sum()
    
    with ProgressBar():
        percent_missing = ((missing_values / dataframe.index.size) * 100).compute()
        
    return percent_missing

check_for_missing(x_train_df)

In [ ]:
def nan_filter(dataframe):
    """ Filter out NaN
    :param data: dataframe
    :return: dataframe
    
    """
    filtered = dataframe.dropna(how='all', subset=None, thresh=None).compute()

    return filtered

filtered = nan_filter(x_train_df)
percent_missing(filtered)

In [ ]:
def drop_columns(dataframe):
    """ Drop columns which have more than 50% missing
    :param data: dataframe
    :return: dataframe
    
    """
    columns_to_drop = list(percent_missing[percent_missing >= 50].index)
    data = dataframe.drop(columns_to_drop, axis=1)
    return data

In [ ]:
def drop_rows(dataframe):
    """ Drop rows if more than 0 and less than 5% missing 
    :param data: dataframe
    :return: dataframe
    
    """
    rows_to_drop = list(percent_missing[(percent_missing > 0) & (percent_missing < 5)].index)
    data_clean = dataframe.dropna(subset=rows_to_drop)

In [ ]:
def impute_rows(dataframe,feature):
    """ Impute rows if more than 5% and less than 50% missing
    :param data: dataframe, feature
    :return: dataframe
    
    """
    value = median[feature]
    df_impute[feature] = df[feature].fillna(value)
    return df_impute

In [ ]:
# Print the size of a feature
print (x_train_df.count().compute())


In [ ]:
# Build model and train

# Split data into train and validation

# JØRGEN KAN BEGYNNE BYGGE DETTE

from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from dask.diagnostics import ProgressBar

bern = BernoulliNB()
with ProgressBar():
    bern.fit(x_train_df, y_train_df)